In [1]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [2]:
product_type = "frozen"
num_time_periods = 7
test_article = '468a73f3'

In [3]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [4]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [5]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [6]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [7]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [8]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [9]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [10]:
shelf_life[test_article] = 2
cu_per_tu[test_article] = 2
for key in demand[test_article].keys():
    demand[test_article][key] = 0
demand[test_article][time_indexes[0]] = 5
demand[test_article][time_indexes[1]] = 6
# demand[test_article][time_indexes[2]] = 1
# demand[test_article][time_indexes[3]] = 1
demand[test_article]

{0: 5, 1: 6, 2: 0, 3: 0, 4: 0, 5: 0}

In [11]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_indexes, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Xit1t2
orders_disaggregated = m.addVars(items, time_indexes, time_indexes, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, t] for item in items for t in time_indexes)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, t] for item in items for t in time_indexes)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[t] for t in time_indexes)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, t] for item in items for t in time_indexes)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for t in time_indexes:
        if(t==0):
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                -
                demand[item][t]
                -
                clearances[item, t]
                >=
                storage_used[item, t]
            )
        else:
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                +
                storage_used[item, t-1]
                -
                demand[item][t]
                -
                clearances[item, t]
                >=
                storage_used[item, t]
            )

# inventory volume constraint
for t in time_indexes:
    if(t==0):
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] * 
                (
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )
    else:
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] *
                (
                    storage_used[item,t-1]
                    +
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )

# min/max constraints (linking too)
for item in demand.keys():
    for t in time_indexes:
        m.addConstr(
            orders[item, t]
            >=
            minimum_order_quantity_tu[item] * ordered_boolean[item, t]
        )
        #TODO: max constraint

for item in demand.keys():
    for t in time_indexes:
        life = shelf_life[item]
        if(t >= life):
            m.addConstr(
                clearances[item, t]
                >=
                gp.quicksum(
                    cu_per_tu[item] * orders[item, t1] for t1 in time_indexes[:t-life]
                )
                -
                gp.quicksum(
                    demand[item][t1] for t1 in time_indexes[:t]
                )
                -
                gp.quicksum(
                    clearances[item, t1] for t1 in time_indexes[:t-1]
                )
            )

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-21
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4174 rows, 25146 columns and 14950 nonzeros
Model fingerprint: 0x10efd2af
Variable types: 0 continuous, 25146 integer (2514 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+01]
  Objective range  [1e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 5610.5000000
Presolve removed 2417 rows and 21708 columns
Presolve time: 0.12s
Presolved: 1757 rows, 3438 columns, 8260 nonzeros
Found heuristic solution: objective 5275.2000000
Variable types: 0 continuous, 3438 integer (413 binary)

Root relaxation: objective 4.632768e+03, 2245 iterations, 0.03 seconds (0.00 work units)

    Nodes    |

In [12]:
for t in time_indexes:
      print(clearances[test_article,t])

<gurobi.Var C7548 (value -0.0)>
<gurobi.Var C7549 (value 1.0)>
<gurobi.Var C7550 (value -0.0)>
<gurobi.Var C7551 (value -0.0)>
<gurobi.Var C7552 (value -0.0)>
<gurobi.Var C7553 (value 0.0)>


In [13]:
for t in time_indexes:
    print(t)
    print("Order: ", orders[test_article,t].X)
    print("Demand: ", demand[test_article][t])
    print("Storage: ", storage_used[test_article,t].X) 
    print()


0
Order:  6.0
Demand:  5
Storage:  7.0

1
Order:  -0.0
Demand:  6
Storage:  -0.0

2
Order:  -0.0
Demand:  0
Storage:  -0.0

3
Order:  -0.0
Demand:  0
Storage:  -0.0

4
Order:  -0.0
Demand:  0
Storage:  -0.0

5
Order:  -0.0
Demand:  0
Storage:  -0.0

